In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from alpacha import AlphcaAPI

api = AlphcaAPI(api_key="PK3WDIKCFF4ZPKWMG5QK25QSOS", api_secret="6oessnea2UcEfgLjA49zD5Mm6mCNtgEUG2DguhWHjn2p")
account_info = api.get_account_info()

ticker_symbol = "AAPL"

end_date = pd.Timestamp.today().date().isoformat()
start_date = (pd.Timestamp.today() - pd.Timedelta(days=1825)).date().isoformat()

stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

stock_data.tail()

In [ ]:
# Data Preprocessing
stock_data['sma_50'] = stock_data['Close'].rolling(window=50).mean()
stock_data['sma_200'] = stock_data['Close'].rolling(window=200).mean()

In [ ]:
stock_data.tail()

In [ ]:
# Generate buy/sell signals
stock_data['signal'] = 0
buy_signals = (stock_data['sma_50'] > stock_data['sma_200']) & (stock_data['sma_50'].shift(1) <= stock_data['sma_200'].shift(1))
stock_data.loc[buy_signals, 'signal'] = 1

sell_signals = (stock_data['sma_50'] < stock_data['sma_200']) & (stock_data['sma_50'].shift(1) >= stock_data['sma_200'].shift(1))
stock_data.loc[sell_signals, 'signal'] = -1

stock_data.tail()

In [ ]:
position = 0
capital = 10000

for i in range(len(stock_data)):
    if stock_data['signal'].iloc[i] == 1 and position != 1:
        position = 1
        capital -= stock_data['Close'].iloc[i]
        print(f"Buy at {stock_data['Close'].iloc[i]} on {stock_data.index[i]}")

    elif stock_data['signal'].iloc[i] == -1 and position != -1:
        position = -1
        capital = position * stock_data['Close'].iloc[i]
        print(f"Sell at {stock_data['Close'].iloc[i]} on {stock_data.index[i]}")
        

In [ ]:
latest_row = stock_data.iloc[[-1]]

if latest_row['signal'].item() == 1:

    order = api.place_order(ticker_symbol, 10, "buy")
    print("Trade executed:", order)
else:
    print("No buy signal on the latest date.")